In [1]:
import numpy as np 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy
from PIL import Image
import glob 


In [2]:
import pandas as pd

df = pd.read_excel ('./multilabels/LandUse_Multilabeled.xlsx')
df_label = np.array(df)



c:\users\karam\anaconda3\envs\pysyft2\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
class_names =  np.array(["airplane","bare-soil","buildings","cars","chaparral","court","dock","field","grass","mobile-home","pavement","sand","sea","ship","tanks","trees","water"])


In [4]:
largestxor = 0 
largestij = (0,0)

for i in range(1,17):
    for j in range(i+1,18):
        #colnand = np.sum(np.logical_not(np.logical_and(df_label[:,i], df_label[:,j])))
        colxor = np.sum(np.logical_xor(df_label[:,i].astype(bool) , df_label[:,j].astype(bool) )) -  np.sum(np.logical_and(df_label[:,i], df_label[:,j]))
        #print(i,j, colxor, colnand)
        if colxor >= largestxor and np.sum(df_label[:,i]) >=700 and np.sum(df_label[:,j])>= 700 :
            largestxor = colxor
            largestij = (i,j)
print(largestxor,class_names[largestij[0]-1], class_names[largestij[1]-1], largestij)

674 bare-soil cars (2, 4)


In [5]:
def uncor_selecter(nr_label = 4,min_img = 300):
    """retrun a list with the least correlated labels """
    image_perlabel = np.sum(df_label[:,1:],axis= 0)
    biggest_label =np.where(np.any([image_perlabel > min_img],axis=0))[0]
    #print(biggest_label, image_perlabel[biggest_label])

    selected_list = [] 
    allcor_lost = np.array([0,0,0])
    for i in range(0,len(biggest_label)-1):
        it = biggest_label[i]
        for j in range(i+1,len(biggest_label)):
            jt = biggest_label[j]

            colxor = np.sum(np.logical_xor(df_label[:,it].astype(bool) , df_label[:,jt].astype(bool) )) -  np.sum(np.logical_and(df_label[:,it], df_label[:,jt]))
            allcor_lost = np.vstack((allcor_lost, np.array([colxor,it,jt]))) 
    sorted_list = allcor_lost[allcor_lost[:,0].argsort()]
    selected_list.append(sorted_list[-1,1])
    selected_list.append(sorted_list[-1,2])
    #print(sorted_list, selected_list)        

    while len(selected_list)<nr_label:
        biggest_label = np.setdiff1d(biggest_label,np.array(selected_list))
        largestxor = 0 
        largestind = 0
        for i in biggest_label:
            overall_xor = 0 
            for j in (selected_list):
                overall_xor += np.sum(np.logical_xor(df_label[:,i].astype(bool) , df_label[:,j].astype(bool) )) -  np.sum(np.logical_and(df_label[:,i], df_label[:,j]))

            if overall_xor >= largestxor:
                largestxor = overall_xor
                largestind = i

        selected_list.append(largestind)
    
    return selected_list


trylist = uncor_selecter()
print(type(trylist), trylist)
            
            


<class 'list'> [1, 2, 3, 8]


In [6]:
import random
def sampler_split_for_client(cdata, idxs, nr_client= 4, minimum_skew_percentage = .4):
    selected_labels = uncor_selecter(nr_client,300)
    
    splitlists = []
    for sb in selected_labels:
        splitlists.append([])
        
    
    for i in idxs:
        nplabel = cdata.__getlabel__(i)
        #nplabel = label.numpy()
        
        if np.any(nplabel[selected_labels] == 1):
            if random.random() < minimum_skew_percentage:
                
                flip = np.random.randint(np.sum(nplabel[selected_labels] == 1)) 
                mask = np.where(nplabel[selected_labels] == 1)[0][flip]
                splitlists[mask].append(i)
            
            else:
                flip = np.random.randint(nr_client) 
                splitlists[flip].append(i)
                    
        else:
            flip = np.random.randint(nr_client) 
            splitlists[flip].append(i)

    
    for alist in splitlists:
        print(len(alist))
    return splitlists
    


In [7]:
from torch.utils.data import Dataset
from natsort import natsorted

class CustomDataSet(Dataset):
    def __init__(self, main_dir, transform, labelmat):
        self.main_dir = main_dir
        self.transforms = transform
        self.all_imgs = glob.glob(os.path.join(main_dir, '**/*.tif'), recursive=True)
        self.total_imgs = natsorted(self.all_imgs)
        self.xlabels = labelmat
        
    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        #print(idx,len(self.total_imgs))
        img_loc = self.total_imgs[idx]
        #print(img_loc)
        imagebaselabel = os.path.splitext(os.path.basename(img_loc))[0]
        label = self.xlabels[np.where(self.xlabels[:,0] == imagebaselabel),1:].reshape(17).astype(np.int64)
        #print(label,label.shape)
        tensor_label =  torch.from_numpy(label)
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transforms(image)
        return tensor_image, tensor_label
    
    def __getlabel__(self, idx):
        
        img_loc = self.total_imgs[idx]
        #print(img_loc)
        imagebaselabel = os.path.splitext(os.path.basename(img_loc))[0]
        label = self.xlabels[np.where(self.xlabels[:,0] == imagebaselabel),1:].reshape(17).astype(np.int64)
        
        return label

In [8]:


data_dir = './UCMerced_LandUse/images'
def load_split_train_test(datadir, labelmat,valid_size = .2):
    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    test_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    train_data = CustomDataSet(datadir, transform=train_transforms, labelmat=labelmat)
    test_data = CustomDataSet(datadir, transform=train_transforms, labelmat=labelmat)

    indices = list(range(2100))
    split = int(np.floor(valid_size * 2100))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    
    
    lists = sampler_split_for_client(train_data, train_idx,3,.4)
    
    return lists
splitlist  = load_split_train_test(data_dir, df_label,.2)


554
506
620
